# Test stuff for PFN model

Classify pions, photons, and **scalars**

In [2]:
# Add import paths
import sys
sys.path.append("..")

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import datetime as dt
from utils import print_gpu_memory

# Make tensorflow not use too much memory
print(f"Importing tensorflow...")
import tensorflow as tf
print(f"Enabling memory growth...")
gpus = tf.config.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

disk_dir = "/usatlas/atlas01/atlasdisk/users/atlas_wifeng"

Importing tensorflow...
Enabling memory growth...


In [3]:
# Get data
from data import get_data
(X_train, X_val, X_test,
 Y_train, Y_val, Y_test) = get_data("scalar_test/all_jets_point_cloud.npz")

print(X_train.shape, Y_train.shape)

(240000, 960, 4) (240000, 3)


In [4]:
# Build model
from model import PFN
Phi_sizes = [256, 256, 256, 128, 128, 128]
F_sizes = [256, 256, 256, 128, 128, 128]

_, n_particles, n_features = X_train.shape
model = PFN(
    n_features=n_features,
    n_particles=n_particles,
    n_outputs=Y_train.shape[1],
    Phi_sizes=Phi_sizes,
    F_sizes=F_sizes
)

In [5]:
from collections import defaultdict
history = defaultdict(list)

In [6]:
# Utility functions for training
from train_model import train_model

def train_iteration(lr, epochs):
    fit_history = train_model(
        model=model, 
        data=(X_train, X_val, Y_train, Y_val),
        lr=lr,
        epochs=epochs
    )
    print(f"\nSaving history...")
    for key in ["loss", "val_loss", "accuracy", "val_accuracy"]:
        history[key].extend(fit_history.history[key])
    return fit_history

def save_model(name):
    model_dir = f"{disk_dir}/photon-jet/models/pfn"
    cur_date = dt.datetime.now().strftime("%Y-%m-%d")
    model.save(f"{model_dir}/{name}_{cur_date}")

In [7]:
print_gpu_memory()

GPU memory usage
  current: 1.75 MB
  peak:    2.0 MB


In [8]:
print(f"=== Training [{dt.datetime.now()}] ===")
train_iteration(lr=2e-4, epochs=30)

=== Training [2023-06-07 20:12:13.511548] ===
Epoch 1/30
2400/2400 [==============================] - 94s 36ms/step - loss: 2.8511 - accuracy: 0.3363 - val_loss: 1.0982 - val_accuracy: 0.3330
Epoch 2/30
2400/2400 [==============================] - 86s 36ms/step - loss: 1.1433 - accuracy: 0.3765 - val_loss: 1.0536 - val_accuracy: 0.4084
Epoch 3/30
2400/2400 [==============================] - 86s 36ms/step - loss: 1.0433 - accuracy: 0.4416 - val_loss: 1.0351 - val_accuracy: 0.4453
Epoch 4/30
2400/2400 [==============================] - 86s 36ms/step - loss: 1.0368 - accuracy: 0.4477 - val_loss: 1.0399 - val_accuracy: 0.4381
Epoch 5/30
2400/2400 [==============================] - 86s 36ms/step - loss: 1.0649 - accuracy: 0.4520 - val_loss: 1.0174 - val_accuracy: 0.4523
Epoch 6/30
2400/2400 [==============================] - 86s 36ms/step - loss: 0.9774 - accuracy: 0.4904 - val_loss: 0.9398 - val_accuracy: 0.5240
Epoch 7/30
2400/2400 [==============================] - 86s 36ms/step - loss: 

In [9]:
print(f"=== Training [{dt.datetime.now()}] ===")
train_iteration(lr=2e-5, epochs=30)

=== Training [2023-06-07 20:55:42.446690] ===
Epoch 1/30
2400/2400 [==============================] - 91s 36ms/step - loss: 0.2041 - accuracy: 0.9216 - val_loss: 0.2215 - val_accuracy: 0.9137
Epoch 2/30
2400/2400 [==============================] - 86s 36ms/step - loss: 0.1876 - accuracy: 0.9294 - val_loss: 0.1828 - val_accuracy: 0.9298
Epoch 3/30
2400/2400 [==============================] - 86s 36ms/step - loss: 0.1765 - accuracy: 0.9335 - val_loss: 0.1686 - val_accuracy: 0.9372
Epoch 4/30
2400/2400 [==============================] - 86s 36ms/step - loss: 0.1674 - accuracy: 0.9371 - val_loss: 0.1694 - val_accuracy: 0.9362
Epoch 5/30
2400/2400 [==============================] - 86s 36ms/step - loss: 0.1599 - accuracy: 0.9405 - val_loss: 0.1628 - val_accuracy: 0.9383
Epoch 6/30
2400/2400 [==============================] - 86s 36ms/step - loss: 0.1526 - accuracy: 0.9435 - val_loss: 0.1449 - val_accuracy: 0.9464
Epoch 7/30
2400/2400 [==============================] - 86s 36ms/step - loss: 

In [10]:
save_model("bigger_model")

TypeError: save_model() takes 0 positional arguments but 1 was given

In [ ]:
print(f"=== Training [{dt.datetime.now()}] ===")
train_iteration(lr=2e-6, epochs=30)

### Evaluate model

In [ ]:
from matplotlib import pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(10, 4))
ax1, ax2 = axs

ax1.plot(history["loss"])
ax1.plot(history["val_loss"])
ax1.legend(["loss", "val_loss"])
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss");

ax2.plot(history["accuracy"])
ax2.plot(history["val_accuracy"])
ax2.legend(["loss", "val_accuracy"])
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Accuracy");

In [ ]:
# Test model
from test_model import test_model, plot_cm
test_accuracy, cm = test_model(model, (X_test, Y_test))
print(f"Testing accuracy: {test_accuracy}")

plot_cm(cm, ["pion", "photon", "axion1"])